In [126]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import os

CLIENT_SECRET_FILE = 'client_secret3.json'
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']

flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_console()
youtube = build('youtube', 'v3', credentials=credentials)

#https://console.developers.google.com/iam-admin/quotas?project=api-project-233696733563
#https://console.developers.google.com/apis/api/youtube.googleapis.com/quotas?project=isentropic-now-275613


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=343081922184-9kpd3mgj9ljigekci6et13eh50mljmih.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=x07zReGC31YYOpkbFLwHCMnqG1FoWE&prompt=consent&access_type=offline
Enter the authorization code: 4/zgFjtchCk9zzRS1dmiU7F1u9Y3DC_noTCc0eZRiJm6_QKgXxFoOrzR4


In [ ]:
def get_playlist(token):
    request = youtube.playlists().list(
            part="snippet,contentDetails",
            mine=True,
            maxResults=50,
            pageToken=token
        )
    response = request.execute()
    return response
    

def get_video_list(token,vtype):
    request = youtube.search().list(
        part="snippet",
        forMine=True,
        type=vtype,
        maxResults=50,
        order="date",
        pageToken=token
    )
    response = request.execute()
    #print(response)
    return response

def get_playlist_videos(playlistid, token):
    request = youtube.playlistItems().list(
            playlistId=playlistid,
            part="contentDetails,status,snippet",
            maxResults=50,
            pageToken=token
        )
    response = request.execute() 
    #f=response['items'][0]
    #print(f)
    return response

def show_videolist():
    my_uploaded_video_playlist='UUC5hntNsVE7rQRYqqv_ON0g'
    data =pd.DataFrame()
    count =0
    size=50
    max_item=1
    nextPageToken =None
    while(count*size <= max_item) :
        rs=get_playlist_videos(my_uploaded_video_playlist,nextPageToken)
        #rs=get_video_list(nextPageToken,'video')
        max_item=int(rs['pageInfo']['totalResults'])
        nextPageToken= rs.get('nextPageToken')
        val=[{'id':d['contentDetails']['videoId'], 'title' :d['snippet']['title'], 'status':d['status']['privacyStatus'] }  for d in rs['items']] 
        print(len(val),max_item,nextPageToken)
        data=data.append(val)
        count+=1  
    return data

def show_youtube_playlist():
    data =pd.DataFrame()
    count =0
    size=50
    max_item=1
    nextPageToken =None
    while(count*size <= max_item) :
        rs=get_playlist(nextPageToken)
        max_item=int(rs['pageInfo']['totalResults'])
        #print(rs)
        nextPageToken= rs.get('nextPageToken')
        val=[{ 'playlist_id': item['id'], 'playlist_title': item['snippet']['title'] ,'video_count': item["contentDetails"]["itemCount"]} for item in rs['items']]
        data = data.append(val)
        count+=1  
    return data

def create_playlist(title):
    request = youtube.playlists().insert(
        part="snippet,status",
        body={
          "snippet": {
            "title":  title,
            "description":  title,
            "tags": [
              "ML,AI "          
            ],
            "defaultLanguage": "en"
          },
          "status": {
            "privacyStatus": "Unlisted"
          }
        }
    )
    response = request.execute()
    if(response is not None and 'id' in response) :
        print("playlist added successfully with id :{0}".format(response["id"]))
    else:
          print("failed to create playlist ")
          print(response)
        
def add_playlist_item(playlist_id, video_id):
    request = youtube.playlistItems().insert(
        part="snippet",
        body={
          "snippet": {
            "playlistId": playlist_id,
            "resourceId": {
              "kind": "youtube#video",
              "videoId": video_id
            }
          }
        }
    )
    response = request.execute()
    if(response.get('id') is not None) :
        print('item added ' + str(id))
    #print(response)

def UpdateVideo(id):
    request = youtube.videos().update(
        part="status",
        body={
          "id": id,
          "status": {
            "privacyStatus": "Unlisted"
          }
        }
    )
    response = request.execute()
    if(response.get('id') is not None):
        print('video updated - ' +response.get('id'))
    else :
        print(response)    
    
    
def delete_playlist(playlist_id):
     request =  youtube.playlists().delete(id=playlist_id)
     response = request.execute()
     print(response)
      

In [ ]:
def get_video_files(fullpath):
    filter_extension=".ts"
    data = [ { 'course_type' :path.split('\\')[6].split('(')[0][-1],
               'playlist' :"ML & AI |" + '|'.join(path.split('\\')[6:]) , \
               'file_name':os.path.splitext(file)[0],  "path" : path} \
            for path, subdirs, files in os.walk(fullpath) for file in files if file.endswith(filter_extension)] 
    df= pd.DataFrame(data)
    return df

def create_playlist_data(fullpath, ptype=None):
    df =get_video_files(fullpath)
    if(ptype is not None) :
        df = df[df['course_type']== str(ptype)]     
    groups=df.groupby('playlist')
    return list(groups.groups.keys())



In [ ]:

#add videolist item
#add_playlist_item(youtube,"PL9tQy71sPLeHoB-g1H08J1hR8ZsXWzz7Z","05yT5rR-XsM")



#get_list(None,'video')
#create playlist

#create_playlist(youtube,"")

#delete_playlist()




In [ ]:
##video listing
rs= show_videolist()
rs.to_csv('youtube_videlist.csv', index=False)
len(rs)

In [ ]:
##create video list from dir path
df=get_video_files(r"C:\Users\Ashwini\Downloads\PGDM\Video")
df.to_csv('dir_file_status.csv', index=False)


In [ ]:
##show playlists
df=show_youtube_playlist()
df.to_csv('youtube_playlist.csv', index=False)

In [ ]:
# create playlist by this on youtube 

dt=create_playlist_data(r"C:\Users\Ashwini\Downloads\PGDM\Video",3)

print(len(dt))

#resume from 10
for g in dt[10:]: #remaining
    print(g)
    #create_playlist(g)




In [ ]:
##get file to map youtube video with playlist

file_status = pd.read_csv('dir_file_status.csv')
youtube_videlist = pd.read_csv('youtube_videlist.csv')
youtube_playlist= pd.read_csv('youtube_playlist.csv')


del file_status['path']
file_status['file_name']= file_status['file_name'] \
.str.replace('-',' ').str.replace('(','').str.replace(')','').str.replace('.',' ')


print(len(file_status['file_name']),len(youtube_videlist['title']))

rs1=file_status.merge(youtube_videlist,left_on='file_name', right_on='title', how='left')
rs2=rs1.merge(youtube_playlist,left_on='playlist', right_on='playlist_title', how='left')
rs2.to_csv('result.csv', index=False)

#file_status['file_name']==youtube_videlist['title']

#rs1[rs1['id'].isnull()].to_csv('not_uploaded.csv')


In [ ]:
##LINK VIDEO WITH PLAYLIST

rs = pd.read_csv('result.csv')

rs=rs[rs['course_type']==2]

print(len(rs))

rs.sort_values(by=['title'], inplace=True)

rs= rs[152:]

for i,r in rs.iterrows(): #116
    print(i,r['playlist_id'],r['id'],r['title'])
    #add_playlist_item(r['playlist_id'],r['id'])
    
    
#rs.isnull().sum()
 #213

In [128]:
#update youtube video access level
rs = pd.read_csv('youtube_videlist.csv')

rs=rs[rs['status']=='private']
print(len(rs))

rs= rs[20:]

for i,r in rs.iterrows(): #116
    print(i,r['id'])
    #UpdateVideo(r['id']) 
    
#last 674   KHBJeAhUnU0

94
12 6UaDL0RsoJQ
video updated - 6UaDL0RsoJQ
13 IowmTOBAZwk
video updated - IowmTOBAZwk
14 nTmSYePuS3w
video updated - nTmSYePuS3w
15 GlSwt86byQM
video updated - GlSwt86byQM
16 7Nfn6Ocy9X8
video updated - 7Nfn6Ocy9X8
17 C_gi4q-iztg
video updated - C_gi4q-iztg
18 3MXMpzxJpmw
video updated - 3MXMpzxJpmw
19 n_8_c4A3rlY
video updated - n_8_c4A3rlY
20 pKufqKtR_4U
video updated - pKufqKtR_4U
21 smq3jByYJpw
video updated - smq3jByYJpw
